## **Load Data**
---------

**These descriptions provide an explanation of each column in the dataset:**

- **person_age** : Age
- **person_income** : Annual income
- **person_home_ownership** : Type of home ownership [Rent, Mortgage, Own, Other]
- **person_emp_length** : Employment length (in years)
- **loan_intent** : Intent behind loan
- **loan_grade** : Loan grade based on credit [A-G]
- **loan_amnt** : Loan amount
- **loan_int_rate** : Interest rate for the loan
- **loan_status** : Loan status [0 is non default 1 is default]
- **loan_percent_income** : Percentage of income
- **cb_person_default_on_file** : Historical default [Y, N]
- **cb_preson_cred_hist_length** : Credit history length

In [11]:
#load library and configuration
import pandas as pd 
import sys

#append a specific path to the system path
sys.path.append("../src")

In [12]:
#import the 'utils' module which contains utility functions
import utils

ModuleNotFoundError: No module named 'utils'

In [13]:
#load configuration or data using 'config_load()' function from the 'utils' module
config_data = utils.config_load()
#display the loaded configuration data
config_data

NameError: name 'utils' is not defined